In [ ]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import os
from itertools import combinations

import numpy as np
import pickle
import data

import importlib
importlib.reload(data)

from data import update_expl_data, update_target_data, food_categories, preprocess_expl
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

import json

In [ ]:
"""
Set sample rate. In this notebook, all data will be resampled at the chosen frequency.
'MS' : Monthly (Month Start)
'W' : Weekly
'D' : Daily
"""

year_period = {'MS': 12, 'W': 52, 'D': 365}
frequency = 'MS'
one_year = year_period[frequency]
output_path = "./output/ensemble_TEST/"
if not os.path.exists(output_path):
    if not os.path.exists("./output"):
        os.mkdir("./output")
    os.mkdir(output_path)

In [ ]:
"""
Load food CPI data from January 1986 to the most recently available data.
"""

foodprice_categories = pd.read_csv("./foodprice_categories.txt", sep='\n', header=None)[0].to_list()
foodprice_df = pd.read_csv("./all_data.csv", index_col=0)
foodprice_df = foodprice_df.set_index(pd.DatetimeIndex(foodprice_df.index))
foodprice_df

In [ ]:
all_experiment_names = pd.Series(list(os.walk("./output"))[0][1])
all_experiment_names = all_experiment_names[~all_experiment_names.str.contains('ensemble')]
all_experiment_names

In [ ]:
experiment_names = all_experiment_names.copy()
report_sim_dates = pd.read_csv("./reportsimdates.txt", sep='\n', header=None)[0].to_list()
sim_train_dates = {}
sim_valid_dates = {}

for date in report_sim_dates:
    sim_train_dates[date] = foodprice_df.index[foodprice_df.index <= date]
    sim_valid_dates[date] = foodprice_df.index[(foodprice_df.index > date) & (foodprice_df.index <= (pd.to_datetime(date) + pd.DateOffset(months=18)))]

In [ ]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

metrics = {
    'r2_score': r2_score,
    'mae': mean_absolute_error,
    'mape': mean_absolute_percentage_error,
    'mse': mean_squared_error,
    'rmse': rmse
}

In [ ]:
def get_forecast_df(food_category, experiment_names, experiment_date):

    forecasts = []

    for ex_name in experiment_names:
        fc_df = pd.read_csv(f"./output/{ex_name}/forecasts_{experiment_date}.csv")
        if "REF_DATE" in fc_df.columns:
            fc_df = fc_df.set_index("REF_DATE")
        else:
            fc_df = fc_df.set_index("Unnamed: 0")
        fc_df.index = pd.DatetimeIndex(fc_df.index)
        fc_series = fc_df[food_category]
        fc_series = fc_series.rename(ex_name)
        forecasts.append(fc_series)

    return pd.concat(forecasts, axis=1)

In [ ]:
get_forecast_df("Meat", experiment_names, "2017-07-01")

In [ ]:
def eval_mean_forecast(foodprice_df, food_category, experiment_date, valid_dates):
      
    valid_series = foodprice_df.loc[valid_dates][food_category]                                 # Select the validation data. 
    all_forecasts = get_forecast_df(food_category, experiment_names, experiment_date)       # Produce the validation period forecast.
    mean_forecast = all_forecasts.mean(axis=1)
    std_forecast =  all_forecasts.std(axis=1)

    valid_metrics = {metric_name: metric_fn(y_true=valid_series,                        # Compute validation metrics. 
                                      y_pred=mean_forecast) for metric_name, metric_fn in metrics.items()}

    return valid_series, mean_forecast, std_forecast, valid_metrics

## Create ensemble with all forecasts and evaluate mean forecasts

In [ ]:
all_valid_metrics = {}
all_forecasts = {}

for report_sim_date in report_sim_dates:

    food_forecasts = {}
    food_scores = {}

    for category in food_categories:
        actual, fc, fc_std, scores = eval_mean_forecast(foodprice_df, category, date, sim_valid_dates[date])
        food_scores[category] = scores
        food_forecasts[category] = fc
        
    all_valid_metrics[report_sim_date] = food_scores
    all_forecasts[report_sim_date] = food_forecasts

In [ ]:
valid_metrics_concat = {}

all_valid_metrics.keys()

for report_date, valid_scores in all_valid_metrics.items():
    valid_metrics_concat[report_date] = pd.DataFrame(valid_scores).T
index = valid_metrics_concat[report_date].index
columns = valid_metrics_concat[report_date].columns
scores = [df.values for date, df in valid_metrics_concat.items()]
mean_scores = pd.DataFrame(np.array(scores).mean(axis=0), index=index, columns=columns)
# mean_scores.to_csv(f"{output_path}/mean_fc_valid_metrics.csv")
mean_scores

## Visualize All Forecasts

In [ ]:
food_col_stats = {}

for category in food_categories:

    fig, ax = plt.subplots(figsize=(12,4))
    date_stats = []

    for d_index, date in enumerate(report_sim_dates):

        all_forecasts = get_forecast_df(category, experiment_names, date)
        all_forecasts = all_forecasts.loc[all_forecasts.index >= pd.to_datetime(date) + pd.DateOffset(months=6)]
        all_forecasts = all_forecasts.loc[all_forecasts.index < pd.to_datetime(date) + pd.DateOffset(months=18)]
        all_forecasts = all_forecasts.assign(ensemble_mean=all_forecasts.mean(axis=1))

        ax.scatter(foodprice_df[category].index[-240:], foodprice_df[category].iloc[-240:], color='black', s=1)

        col_stats = {}

        for index, col in enumerate(all_forecasts):

            # Collect some stats
            y_pred = all_forecasts[col]
            y_true = foodprice_df[category].loc[y_pred.index]
            col_stats[col] = y_pred - y_true

            if d_index < 1:
                ax.plot(all_forecasts[col], color=f"C{index}", label=col)
            else:
                ax.plot(all_forecasts[col], color=f"C{index}")

        date_stats.append(pd.DataFrame(col_stats))
    
    food_col_stats[category] = pd.concat(date_stats, axis=0)

    plt.title(category)
    plt.legend(bbox_to_anchor=(1,1))
    plt.grid()
    # plt.savefig(f"./report_output/ensemble_raw_{category}.svg", bbox_inches='tight')
    plt.show()


In [ ]:
from scipy.stats import ttest_1samp

all_t_stats = []

for col in food_col_stats:
    for exp_col in food_col_stats[col]:

        errors = food_col_stats[col][exp_col].dropna(axis=0).values
        t_stat, p_value = ttest_1samp(errors, popmean=0)
        # print(col, exp_col, t_stat, p_value)
        all_t_stats.append({"Category": col, "Experiment": exp_col, 't_statistic': t_stat, 'p_value': p_value})

In [ ]:
pd.DataFrame(all_t_stats)

## Visualize Mean Forecasts

In [ ]:
for category in food_categories:

    fig, ax = plt.subplots(figsize=(12,4))

    for d_index, date in enumerate(report_sim_dates):

        all_forecasts = get_forecast_df(category, experiment_names, date)
        all_forecasts = all_forecasts.loc[all_forecasts.index >= pd.to_datetime(date) + pd.DateOffset(months=6)]
        all_forecasts = all_forecasts.loc[all_forecasts.index < pd.to_datetime(date) + pd.DateOffset(months=18)]

        forecast_min = np.min(all_forecasts, axis=1) 
        forecast_10 = np.percentile(all_forecasts, 10, axis=1)
        forecast_25 = np.percentile(all_forecasts, 25, axis=1)
        forecast_75 = np.percentile(all_forecasts, 75, axis=1)
        forecast_90 = np.percentile(all_forecasts, 90, axis=1)
        forecast_max = np.max(all_forecasts, axis=1) 

        if d_index < 1:
            # ax.plot(all_forecasts.index, all_forecasts.mean(axis=1), color='red', label='Mean Forecast')
            ax.plot(all_forecasts.index, all_forecasts.median(axis=1), color='blue', label='Median Forecast')
            ax.fill_between(all_forecasts.index, forecast_min, forecast_max, alpha=0.1, color='purple', label='Forecast Range')
            ax.fill_between(all_forecasts.index, forecast_10, forecast_90, alpha=0.3, color='purple', label='90th Percentile')
            ax.fill_between(all_forecasts.index, forecast_25, forecast_75, alpha=0.6, color='purple', label='75th Percentile')
        else:
            # ax.plot(all_forecasts.index, all_forecasts.mean(axis=1), color='red')
            ax.plot(all_forecasts.index, all_forecasts.median(axis=1), color='blue')
            ax.fill_between(all_forecasts.index, forecast_min, forecast_max, alpha=0.1, color='purple')
            ax.fill_between(all_forecasts.index, forecast_10, forecast_90, alpha=0.3, color='purple')
            ax.fill_between(all_forecasts.index, forecast_25, forecast_75, alpha=0.6, color='purple')

    ax.scatter(foodprice_df[category].index[-120:], foodprice_df[category].iloc[-120:], color='black', s=1, label="Actual CPI")
    plt.title(category)
    plt.legend(loc='upper left')
    plt.grid()
    # plt.savefig(f"./report_output/ensemble_range_{category}.svg", bbox_inches='tight')
    plt.show()


## Search for the best-validating ensembles per-category over the whole validation period

With the multivariate NeuralProphet models, there are now many model configurations to choose from. Which ones should be combined and ensembled? Let's do this in a completely systematic way and find the configuration that validates best. We'll do this separately for each food price category.

In [ ]:
all_combinations = []

for c_size in range(1, 4):
    for exp in combinations(experiment_names, c_size):
        all_combinations.append(exp)

print(len(all_combinations))

In [ ]:
category_config_scores = {}

for category in food_categories:

    print(f"Computing ensemble scores for {category}")

    config_scores = {}
    config_stddevs = {}

    for config_index, exp_config in enumerate(all_combinations):

        if config_index % 100 == 0:
            print(f"Processing config {config_index + 1} of {len(all_combinations)}...")

        valid_date_errors = []

        for d_index, date in enumerate(report_sim_dates):

            all_forecasts = get_forecast_df(category, exp_config, date)

            all_forecasts = all_forecasts.loc[all_forecasts.index >= pd.to_datetime(date) + pd.DateOffset(months=6)]
            all_forecasts = all_forecasts.loc[all_forecasts.index < pd.to_datetime(date) + pd.DateOffset(months=18)]

            mean_forecast = all_forecasts.mean(axis=1)
            actual_cpi = foodprice_df[category][mean_forecast.index]

            analysis_df = pd.DataFrame({'y': actual_cpi, 'yhat': mean_forecast})
            valid_date_errors.append(mean_absolute_percentage_error(y_true=analysis_df.y, y_pred=analysis_df.yhat))

        mean_mape = np.mean(valid_date_errors)
        std_mape = np.std(valid_date_errors)
        config_scores[str(exp_config)] = mean_mape
        config_stddevs[str(exp_config)] = std_mape
        
    category_config_scores[category] =  pd.DataFrame({'mape': config_scores, 'stddev': config_stddevs})  # pd.Series(config_scores, name='mape').to_frame()

In [ ]:
category_config_scores['Bakery and cereal products (excluding baby food)'].mape.plot(kind='hist', bins=20)

In [ ]:
category_config_scores["Meat"].head(60)

In [ ]:
category_config_scores["Vegetables and vegetable preparations"].sort_values('mape')

In [ ]:
results = {}
for category in food_categories:
    best_exp_config = category_config_scores[category].idxmin()['mape']
    best_exp_config = best_exp_config.replace("'",'').replace(' ', '').strip('()').split(',')
    if '' in best_exp_config:
        best_exp_config.remove('')
    results[category] = best_exp_config
    print(f"{category}\t{best_exp_config}\t{category_config_scores[category]['mape'].min()}\t{category_config_scores[category].sort_values('mape')['stddev'][0]}")

In [ ]:
results

In [ ]:
scores = {}
for food_category in food_categories:
    best_index = category_config_scores[food_category].mape.idxmin()
    scores[food_category] = category_config_scores[food_category].loc[best_index]

In [ ]:
"""
The following scores provide uncertainty ranges for best ensemble
mean forecasts over the simulated report years. For example, if for
Meat we observe a mape of 0.026 and stddev of 0.010, we could say
that over the last 6 years, this methodology's forecasts had an average
error of 2.6% +/- 1.0% standard deviation. Our experiments are designed
to select the category-specific model ensemble that would have had the 
lowest MAPE, on average, if the ensemble had been used over the last 6 years.
"""

results_df = pd.DataFrame(scores).T
results_df["best_config"] = pd.Series(results, name='best_config')
results_df

## Save the ensemble configurations to file

In [ ]:
"""
Save ensemble configurations and scores. 
"""
results_df.to_pickle(f"{output_path}/ensemble_results.pkl")

## Visualize the best ensemble forecasts per category and collect the best configuration MAPEs

In [ ]:
for category in food_categories:

    fig, ax = plt.subplots(figsize=(12,4))
    best_exp_config = category_config_scores[category].idxmin()['mape']
    best_exp_config = best_exp_config.replace("'",'').replace(' ', '').strip('()').split(',')
    best_exp_config = [config for config in best_exp_config if len(config) > 0]

    print(category)
    print(best_exp_config)
    print()

    for d_index, date in enumerate(report_sim_dates):

        all_forecasts = get_forecast_df(category, best_exp_config, date)
        all_forecasts = all_forecasts.loc[all_forecasts.index >= pd.to_datetime(date) + pd.DateOffset(months=6)]
        all_forecasts = all_forecasts.loc[all_forecasts.index < pd.to_datetime(date) + pd.DateOffset(months=18)]

        forecast_min = np.min(all_forecasts, axis=1) 
        forecast_std = np.std(all_forecasts, axis=1)
        forecast_mean = np.mean(all_forecasts, axis=1)
        forecast_max = np.max(all_forecasts, axis=1) 

        if d_index < 1:
            # ax.plot(all_forecasts.index, all_forecasts.mean(axis=1), color='red', label='Mean Forecast')
            ax.plot(all_forecasts.index, all_forecasts.mean(axis=1), color='blue', label='Mean Forecast')
            ax.fill_between(all_forecasts.index, forecast_min, forecast_max, alpha=0.1, color='purple', label='Forecast Range')
            ax.fill_between(all_forecasts.index, forecast_mean - forecast_std, forecast_mean + forecast_std, alpha=0.3, color='purple', label='Std. Dev.')
        else:
            # ax.plot(all_forecasts.index, all_forecasts.mean(axis=1), color='red')
            ax.plot(all_forecasts.index, all_forecasts.mean(axis=1), color='blue')
            ax.fill_between(all_forecasts.index, forecast_min, forecast_max, alpha=0.1, color='purple')
            ax.fill_between(all_forecasts.index,forecast_mean - forecast_std, forecast_mean + forecast_std, alpha=0.3, color='purple')

    ax.scatter(foodprice_df[category].index[-240:], foodprice_df[category].iloc[-240:], color='black', s=1, label="Actual CPI")
    plt.title(category)
    plt.legend(loc='upper left')
    plt.grid()
    # plt.savefig(f"./report_output/ensemble_range_{category}.svg", bbox_inches='tight')
    plt.show()


In [ ]:
food_col_stats = {}

for category in food_categories:

    best_exp_config = category_config_scores[category].idxmin()['mape']
    best_exp_config = best_exp_config.replace("'",'').replace(' ', '').strip('()').split(',')
    best_exp_config = [config for config in best_exp_config if len(config) > 0]

    fig, ax = plt.subplots(figsize=(12,4))
    date_stats = []

    for d_index, date in enumerate(report_sim_dates):

        all_forecasts = get_forecast_df(category, best_exp_config, date)
        all_forecasts = all_forecasts.loc[all_forecasts.index >= pd.to_datetime(date) + pd.DateOffset(months=6)]
        all_forecasts = all_forecasts.loc[all_forecasts.index < pd.to_datetime(date) + pd.DateOffset(months=18)]
        all_forecasts = all_forecasts.assign(ensemble_mean=all_forecasts.mean(axis=1))

        ax.scatter(foodprice_df[category].index[-240:], foodprice_df[category].iloc[-240:], color='black', s=1)

        col_stats = {}

        for index, col in enumerate(all_forecasts):

            # Collect some stats
            y_pred = all_forecasts[col]
            y_true = foodprice_df[category].loc[y_pred.index]
            col_stats[col] = y_pred - y_true

            if d_index < 1:
                ax.plot(all_forecasts[col], color=f"C{index}", label=col)
            else:
                ax.plot(all_forecasts[col], color=f"C{index}")

        date_stats.append(pd.DataFrame(col_stats))
    
    food_col_stats[category] = pd.concat(date_stats, axis=0)

    plt.title(category)
    plt.legend(loc='best')
    plt.grid()
    # plt.savefig(f"./report_output/ensemble_raw_{category}.svg", bbox_inches='tight')
    plt.show()
